# Machine Learning

<b>Recurrent Neural Networks (RNN) with Keras


Recurrent neural networks (RNN) are a class of neural networks that is powerful for modeling sequence data such as time series or natural language. It is prominent in the field of NLP(Natural language processing). Recurrent Neural Network(RNN) are a type of Neural Network where the output from previous step are fed as input to the current step. In traditional neural networks, all the inputs and outputs are independent of each other, but in cases like when it is required to predict the next word of a sentence, the previous words are required and hence there is a need to remember the previous words. Thus RNN came into existence, which solved this issue with the help of a Hidden Layer. The main and most important feature of RNN is Hidden state, which remembers some information about a sequence.

<b>Importing nessassary libraries

In [3]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM,Dense ,Dropout, Bidirectional
from sklearn.preprocessing import MinMaxScaler

In [4]:
from sklearn.preprocessing import OneHotEncoder

<b>Importing the datasets into pandas data frame

In [5]:
#this csv file contains the readings for entire india
data = pd.read_csv(r"District91_TempAnalysis_1980_20.csv") 

In [6]:
#this csv file contains the readings for banglore which was done sepeartaly with a different shape file
banglore_temp = pd.read_csv(r"Temp_Analysis_Bangalore_1980_2020.csv")

## Predicting Bangalore Mean Temperatures

Here we use explicity the banglore dataset for the prediction.

In [332]:
banglore_temp.head()

,DISPLAY_NA,MOVEMENT_I,WARD_NAME,WARD_NO,date,maximum_2m_air_temperature,minimum_2m_air_temperature
0,"Raghva Niwas, 38th Cross Road, 4th T Block Eas...",162,Pattabhiram Nagar,168,1980-01-01T00:00:00,305.679810,288.141510
1,"100 16 A Main Road, Canara Bank Colony, Jayana...",163,Byrasandra,169,1980-01-01T00:00:00,305.679810,288.141510
2,"725-44, TMC Layout, 1st Phase, JP Nagar, Benga...",164,Shakambari Nagar,179,1980-01-01T00:00:00,305.679810,288.141510
3,"State Highway 35, Devasthanagalu, Gunjur Villa...",165,Varthuru,149,1980-01-01T00:00:00,305.614028,288.051396
4,"Bellandur - Doddakannelli Road, Adarsh Palm Re...",166,Bellanduru,150,1980-01-01T00:00:00,305.679810,288.141510


In [333]:
banglore_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31878 entries, 0 to 31877
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   DISPLAY_NA                  31878 non-null  object 
 1   MOVEMENT_I                  31878 non-null  int64  
 2   WARD_NAME                   31878 non-null  object 
 3   WARD_NO                     31878 non-null  int64  
 4   date                        31878 non-null  object 
 5   maximum_2m_air_temperature  31878 non-null  float64
 6   minimum_2m_air_temperature  31878 non-null  float64
dtypes: float64(2), int64(2), object(3)
memory usage: 1.7+ MB


In [7]:
#we group by date to find the mean
banglore_temp=banglore_temp.groupby('date').mean()

In [335]:
banglore_temp.head()

,MOVEMENT_I,WARD_NO,maximum_2m_air_temperature,minimum_2m_air_temperature
date,,,,
1980-01-01T00:00:00,99.5,99.5,305.580205,288.117520
1980-04-01T00:00:00,99.5,99.5,306.536743,292.963571
1980-07-01T00:00:00,99.5,99.5,301.647714,291.215594
1980-10-01T00:00:00,99.5,99.5,301.881074,286.977627
1981-01-01T00:00:00,99.5,99.5,305.691383,287.973688


we calculate the mean in the below cell

In [336]:
col = banglore_temp.loc[: , "maximum_2m_air_temperature":"minimum_2m_air_temperature"]

In [337]:
banglore_temp['mean_temperature'] = col.mean(axis=1)

In [338]:
banglore_temp.head()

,MOVEMENT_I,WARD_NO,maximum_2m_air_temperature,minimum_2m_air_temperature,mean_temperature
date,,,,,
1980-01-01T00:00:00,99.5,99.5,305.580205,288.117520,296.848862
1980-04-01T00:00:00,99.5,99.5,306.536743,292.963571,299.750157
1980-07-01T00:00:00,99.5,99.5,301.647714,291.215594,296.431654
1980-10-01T00:00:00,99.5,99.5,301.881074,286.977627,294.429350
1981-01-01T00:00:00,99.5,99.5,305.691383,287.973688,296.832536


In [339]:
training_set = banglore_temp["mean_temperature"]

In [340]:
training_set.reset_index(drop=True,inplace=True)

In [341]:
training_set.head()

0    296.848862
1    299.750157
2    296.431654
3    294.429350
4    296.832536
Name: mean_temperature, dtype: float64

In [342]:
len(training_set)

161

In [343]:
training_set = np.asarray(training_set)

In [344]:
training_set = training_set.reshape((-1,1))

In [345]:
training_set=np.round_(training_set, decimals=2)

In [367]:
test_set_sample = training_set[-1:-35:-1]
train_set = training_set[0:137]

In [368]:
print(len(train_set))
print(len(test_set_sample))

137
34


We scale the data, for the purpose of feeding it to the model

In [369]:
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(train_set)

Preparing the data to be fed to the model

In [370]:
x_train = []
y_train = []
n_future = 4 # next 4 temperature forecast
n_past = 30 # Past 30  
for i in range(0,len(training_set_scaled)-n_past-n_future+1):
    x_train.append(training_set_scaled[i : i + n_past , 0])     
    y_train.append(training_set_scaled[i + n_past : i + n_past + n_future , 0 ])
x_train , y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0] , x_train.shape[1], 1) )

Making the RNN model, and fiting the data, and training the model

In [372]:
regressor = Sequential()
regressor.add(Bidirectional(LSTM(units=30, return_sequences=True, input_shape = (x_train.shape[1],1) ) ))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 30 , return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 30 , return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 30))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = n_future,activation='linear'))
regressor.compile(optimizer='adam', loss='mean_squared_error',metrics=['acc'])
regressor.fit(x_train, y_train, epochs=800,batch_size=32 )

Epoch 1/800
4/4 [==============================] - 0s 31ms/step - loss: 0.2768 - acc: 0.2596
Epoch 2/800
4/4 [==============================] - 0s 20ms/step - loss: 0.1660 - acc: 0.3269
Epoch 3/800
4/4 [==============================] - 0s 21ms/step - loss: 0.0895 - acc: 0.2500
Epoch 4/800
4/4 [==============================] - 0s 20ms/step - loss: 0.0816 - acc: 0.2885
Epoch 5/800
4/4 [==============================] - 0s 20ms/step - loss: 0.0774 - acc: 0.1923
Epoch 6/800
4/4 [==============================] - 0s 20ms/step - loss: 0.0763 - acc: 0.2212
Epoch 7/800
4/4 [==============================] - 0s 21ms/step - loss: 0.0782 - acc: 0.1923
Epoch 8/800
4/4 [==============================] - 0s 20ms/step - loss: 0.0615 - acc: 0.3462
Epoch 9/800
4/4 [==============================] - 0s 21ms/step - loss: 0.0691 - acc: 0.2692
Epoch 10/800
4/4 [==============================] - 0s 20ms/step - loss: 0.0678 - acc: 0.2500
Epoch 11/800
4/4 [==============================] - 0s 21ms/step - lo

In [375]:
testing_sample_scaled = sc.fit_transform(test_set_sample[0:30])

Predicting for sample test data prepared 

In [376]:
testing_sample_scaled = np.reshape(testing_sample_scaled,(testing_sample_scaled.shape[1],testing_sample_scaled.shape[0],1))

predicted_temperature = regressor.predict(testing_sample_scaled)

In [377]:
predicted_temperature = sc.inverse_transform(predicted_temperature)

The below are the predicted values

In [396]:
print(predicted_temperature[0])

[298.21646 299.73602 297.93655 296.91956]


Now lets see the actual values

In [397]:
acutal_value = (test_set_sample[30:34])

In [398]:
acutal_value

array([[297.59],
       [300.38],
       [296.74],
       [294.47]])

Now finding the error between actual and predicted data

In [399]:
error1 = predicted_temperature[0] - acutal_value[0]

In [400]:
print(error1)

[ 0.62646118  2.14602295  0.34655396 -0.67044434]


The above error indicates that our model performs very well, now lets use it to predict future temperature

<b> Predicting future temperature for next 4 quartely month

In [401]:
testing_sample_scaled = sc.fit_transform(test_set_sample[4:34])

In [402]:
testing_sample_scaled = np.reshape(testing_sample_scaled,(testing_sample_scaled.shape[1],testing_sample_scaled.shape[0],1))

predicted_temperature = regressor.predict(testing_sample_scaled)

In [403]:
predicted_temperature = sc.inverse_transform(predicted_temperature)

In [404]:
print(predicted_temperature[0])

[297.91928 299.58896 297.62598 296.5048 ]


The above are the predicted mean values for the next quarterly months for banglore

## Predicting Kerala Mean temperature

Here we are using the main ERA5 dataset for this prediction. We predict the mean temperature for kerala state

In [184]:
kerala_data = data[data['STATE_UT']=='KERALA']

In [185]:
kerala_data.head()

,DIST91_ID,NAME,STATE_UT,date,maximum_2m_air_temperature,minimum_2m_air_temperature,mean_temperature
72,171.0,WAYANAD,KERALA,1980-01-01,304.328384,286.505651,295.417017
73,170.0,KANNUR,KERALA,1980-01-01,304.485970,292.209197,298.347583
74,172.0,KOZHIKODE,KERALA,1980-01-01,303.993400,292.673536,298.333468
75,176.0,ERNAKULAM,KERALA,1980-01-01,307.160080,293.908322,300.534201
76,173.0,MALAPPURAM,KERALA,1980-01-01,304.929867,290.547602,297.738735


In [186]:
kerala_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6720 entries, 72 to 313351
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   DIST91_ID                   6720 non-null   float64       
 1   NAME                        6720 non-null   object        
 2   STATE_UT                    6720 non-null   object        
 3   date                        6720 non-null   datetime64[ns]
 4   maximum_2m_air_temperature  6720 non-null   float64       
 5   minimum_2m_air_temperature  6720 non-null   float64       
 6   mean_temperature            6720 non-null   float64       
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 420.0+ KB


Calculating the mean

In [187]:
col = kerala_data.loc[: , "maximum_2m_air_temperature":"minimum_2m_air_temperature"]

In [188]:
kerala_data['mean_temperature'] = col.mean(axis=1)

<ipython-input-188-ada6cff453e5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kerala_data['mean_temperature'] = col.mean(axis=1)


In [189]:
kerala_data.head()

,DIST91_ID,NAME,STATE_UT,date,maximum_2m_air_temperature,minimum_2m_air_temperature,mean_temperature
72,171.0,WAYANAD,KERALA,1980-01-01,304.328384,286.505651,295.417017
73,170.0,KANNUR,KERALA,1980-01-01,304.485970,292.209197,298.347583
74,172.0,KOZHIKODE,KERALA,1980-01-01,303.993400,292.673536,298.333468
75,176.0,ERNAKULAM,KERALA,1980-01-01,307.160080,293.908322,300.534201
76,173.0,MALAPPURAM,KERALA,1980-01-01,304.929867,290.547602,297.738735


In [190]:
training_set = kerala_data["mean_temperature"]

In [191]:
training_set.reset_index(drop=True,inplace=True)

In [192]:
training_set.head()

0    295.417017
1    298.347583
2    298.333468
3    300.534201
4    297.738735
Name: mean_temperature, dtype: float64

In [193]:
training_set = np.asarray(training_set)

In [194]:
training_set = training_set.reshape((-1,1))

In [195]:
len(training_set)

6720

In [200]:
training_set=np.round_(training_set, decimals=2)

In [201]:
training_set

array([[295.42],
       [298.35],
       [298.33],
       ...,
       [299.9 ],
       [302.9 ],
       [300.3 ]])

In [202]:
test_set_sample = training_set[-1:-100:-1]
train_set = training_set[0:6621]

In [203]:
len(train_set)

6621

In [204]:
len(test_set_sample)

99

Scaling the dataset for feeding it to the model

In [205]:
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(train_set)

Preparing the data to be fed to the model

In [206]:
x_train = []
y_train = []
n_future = 4 # next 4 days temperature forecast
n_past = 30 # Past 30 days 
for i in range(0,len(training_set_scaled)-n_past-n_future+1):
    x_train.append(training_set_scaled[i : i + n_past , 0])     
    y_train.append(training_set_scaled[i + n_past : i + n_past + n_future , 0 ])
x_train , y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0] , x_train.shape[1], 1) )

Making the RNN model, and fiting the data, and training the model

In [207]:
regressor = Sequential()
regressor.add(Bidirectional(LSTM(units=30, return_sequences=True, input_shape = (x_train.shape[1],1) ) ))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 30 , return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 30 , return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 30))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = n_future,activation='linear'))
regressor.compile(optimizer='adam', loss='mean_squared_error',metrics=['acc'])
regressor.fit(x_train, y_train, epochs=500,batch_size=32 )

Epoch 1/500
206/206 [==============================] - 6s 28ms/step - loss: 0.0345 - acc: 0.2495
Epoch 2/500
206/206 [==============================] - 6s 29ms/step - loss: 0.0176 - acc: 0.2521
Epoch 3/500
206/206 [==============================] - 6s 28ms/step - loss: 0.0157 - acc: 0.2464
Epoch 4/500
206/206 [==============================] - 6s 28ms/step - loss: 0.0142 - acc: 0.2577
Epoch 5/500
206/206 [==============================] - 6s 29ms/step - loss: 0.0137 - acc: 0.2412
Epoch 6/500
206/206 [==============================] - 6s 28ms/step - loss: 0.0130 - acc: 0.2485
Epoch 7/500
206/206 [==============================] - 6s 30ms/step - loss: 0.0125 - acc: 0.2491: 1s - loss
Epoch 8/500
206/206 [==============================] - 6s 29ms/step - loss: 0.0122 - acc: 0.2400:
Epoch 9/500
206/206 [==============================] - 6s 29ms/step - loss: 0.0119 - acc: 0.2523
Epoch 10/500
206/206 [==============================] - 6s 30ms/step - loss: 0.0117 - acc: 0.2498
Epoch 11/500
206/

In [213]:
testing_sample_scaled = sc.fit_transform(test_set_sample[0:30])

Predicting for sample test data prepared

In [214]:
testing_sample_scaled = np.reshape(testing_sample_scaled,(testing_sample_scaled.shape[1],testing_sample_scaled.shape[0],1))

predicted_temperature = regressor.predict(testing_sample_scaled)

In [215]:
predicted_temperature = sc.inverse_transform(predicted_temperature)

The below are the predicted values

In [216]:
predicted_temperature

array([[300.18274, 300.5763 , 300.68668, 298.6599 ]], dtype=float32)

Seeing the actual value to compare

In [217]:
test_set_sample[30:34]

array([[299.37],
       [299.94],
       [300.47],
       [297.62]])

Finding the error

In [218]:
error = predicted_temperature[0] - test_set_sample[30:34][0]

In [452]:
print(error)

[0.81274 0.6363  0.21668 1.0399 ]


As the above error on the test data increase that, our model performs very well, now lets predict the future 

<b>Predicting future temperature with the current data

In [220]:
testing_sample_scaled = sc.fit_transform(test_set_sample[70:100])

In [221]:
testing_sample_scaled = np.reshape(testing_sample_scaled,(testing_sample_scaled.shape[1],testing_sample_scaled.shape[0],1))

predicted_temperature = regressor.predict(testing_sample_scaled)

In [222]:
predicted_temperature = sc.inverse_transform(predicted_temperature)

In [245]:
print(predicted_temperature[0])

[300.33813 301.1022  300.9345  299.91898]


The above values are the predicted mean temperature of Kerala for next 4 months, the values are in kelvin scale

## Predicting INDORE Maximum temperature

Here we are using the main ERA5 dataset for this prediction, here we predict the mean temperature for INDORE district

In [518]:
indore_data = data[data['NAME']=='INDORE']

In [519]:
indore_data.head()

,DIST91_ID,NAME,STATE_UT,date,maximum_2m_air_temperature,minimum_2m_air_temperature
393,205.0,INDORE,MADHYA_PRADESH,1980-01-01T00:00:00,302.225027,283.010331
1047,205.0,INDORE,MADHYA_PRADESH,1980-02-01T00:00:00,307.376971,281.841915
1701,205.0,INDORE,MADHYA_PRADESH,1980-03-01T00:00:00,309.569063,285.828591
2355,205.0,INDORE,MADHYA_PRADESH,1980-04-01T00:00:00,314.550475,292.911056
3009,205.0,INDORE,MADHYA_PRADESH,1980-05-01T00:00:00,315.050293,297.359887


In [520]:
indore_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 480 entries, 393 to 313659
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   DIST91_ID                   480 non-null    float64
 1   NAME                        480 non-null    object 
 2   STATE_UT                    480 non-null    object 
 3   date                        480 non-null    object 
 4   maximum_2m_air_temperature  480 non-null    float64
 5   minimum_2m_air_temperature  480 non-null    float64
dtypes: float64(3), object(3)
memory usage: 26.2+ KB


Preparing the dataset

In [521]:
training_set = indore_data["maximum_2m_air_temperature"]

In [522]:
training_set.reset_index(drop=True,inplace=True)

In [523]:
training_set = np.asarray(training_set)

In [524]:
training_set = training_set.reshape((-1,1))

In [525]:
len(training_set)

480

In [526]:
test_set_sample = training_set[-1:-61:-1]
train_set = training_set[0:420]

In [527]:
len(test_set_sample)

60

Scaling the data

In [528]:
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(train_set)

In [529]:
x_train = []
y_train = []
n_future = 4 # next 4 days temperature forecast
n_past = 30 # Past 30 days 
for i in range(0,len(training_set_scaled)-n_past-n_future+1):
    x_train.append(training_set_scaled[i : i + n_past , 0])     
    y_train.append(training_set_scaled[i + n_past : i + n_past + n_future , 0 ])
x_train , y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0] , x_train.shape[1], 1) )

Building the model

In [531]:
regressor = Sequential()
regressor.add(Bidirectional(LSTM(units=30, return_sequences=True, input_shape = (x_train.shape[1],1) ) ))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 30 , return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 30 , return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 30))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = n_future,activation='linear'))
regressor.compile(optimizer='adam', loss='mean_squared_error',metrics=['acc'])
regressor.fit(x_train, y_train, epochs=300,batch_size=32 )


Epoch 1/300
13/13 [==============================] - 0s 32ms/step - loss: 0.1658 - acc: 0.2610
Epoch 2/300
13/13 [==============================] - 0s 27ms/step - loss: 0.0859 - acc: 0.2842
Epoch 3/300
13/13 [==============================] - 0s 26ms/step - loss: 0.0729 - acc: 0.2481
Epoch 4/300
13/13 [==============================] - 0s 26ms/step - loss: 0.0712 - acc: 0.2661
Epoch 5/300
13/13 [==============================] - 0s 27ms/step - loss: 0.0724 - acc: 0.2041
Epoch 6/300
13/13 [==============================] - 0s 27ms/step - loss: 0.0684 - acc: 0.2558
Epoch 7/300
13/13 [==============================] - 0s 26ms/step - loss: 0.0698 - acc: 0.2584
Epoch 8/300
13/13 [==============================] - 0s 26ms/step - loss: 0.0690 - acc: 0.2661
Epoch 9/300
13/13 [==============================] - 0s 26ms/step - loss: 0.0681 - acc: 0.2119
Epoch 10/300
13/13 [==============================] - 0s 26ms/step - loss: 0.0662 - acc: 0.2429
Epoch 11/300
13/13 [=============================

In [571]:
testing_sample_scaled = sc.fit_transform(test_set_sample[14:34])

Predicting for sample test data

In [572]:
testing_sample_scaled = np.reshape(testing_sample_scaled,(testing_sample_scaled.shape[1],testing_sample_scaled.shape[0],1))

predicted_temperature = regressor.predict(testing_sample_scaled)

In [573]:
predicted_temperature = sc.inverse_transform(predicted_temperature)

Show predicted data

In [574]:
predicted_temperature[0]

array([306.89902, 307.5554 , 307.065  , 305.6764 ], dtype=float32)

Actual values

In [575]:
actual_value = (test_set_sample[34:38])

In [576]:
actual_value

array([[307.522407  ],
       [303.71404966],
       [304.23127174],
       [305.16312889]])

Finding the error

In [577]:
error = predicted_temperature[0] - actual_value[0]

In [578]:
print(error)

[-0.62338967  0.0329824  -0.45740456 -1.8460154 ]


We see that we get good preformance from this model, so now we use it to predict next 4 months maximum temparature

### Predicting Future Temperature with current data

In [579]:
testing_sample_scaled = sc.fit_transform(test_set_sample[31:61])

In [580]:
testing_sample_scaled = np.reshape(testing_sample_scaled,(testing_sample_scaled.shape[1],testing_sample_scaled.shape[0],1))

predicted_temperature = regressor.predict(testing_sample_scaled)

In [581]:
predicted_temperature = sc.inverse_transform(predicted_temperature)

In [582]:
predicted_temperature[0]

array([307.0098 , 306.61124, 304.31796, 303.4949 ], dtype=float32)

The above values are the predicted temperature values for INDORE district for the next 4 months. The temperature values are in kelvin